## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,61.12,81,61,1.01,broken clouds
1,1,Araouane,ML,18.9048,-3.5265,92.89,8,8,7.85,clear sky
2,2,Bambanglipuro,ID,-7.9500,110.2833,76.55,86,98,10.87,overcast clouds
3,3,Lompoc,US,34.6391,-120.4579,69.94,74,20,11.50,few clouds
4,4,Tasiilaq,GL,65.6145,-37.6368,33.96,75,57,3.06,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bambanglipuro,ID,-7.9500,110.2833,76.55,86,98,10.87,overcast clouds
5,5,Arcata,US,40.8665,-124.0828,73.20,66,1,3.00,clear sky
12,12,Neiafu,TO,-18.6500,-173.9833,79.48,53,75,5.75,broken clouds
20,20,Atuona,PF,-9.8000,-139.0333,78.24,72,10,17.60,clear sky
25,25,Katsuura,JP,35.1333,140.3000,70.11,70,12,18.59,few clouds
26,26,Tezu,IN,27.9167,96.1667,77.16,74,69,3.78,broken clouds
41,41,Avarua,CK,-21.2078,-159.7750,78.85,69,75,4.61,broken clouds
44,44,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,87,9.46,overcast clouds
46,46,Ponta Do Sol,PT,32.6667,-17.1000,70.72,70,49,3.40,scattered clouds
48,48,Moerai,PF,-22.4333,-151.3333,76.17,85,70,10.18,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambanglipuro,ID,76.55,overcast clouds,-7.9500,110.2833,
5,Arcata,US,73.20,clear sky,40.8665,-124.0828,
12,Neiafu,TO,79.48,broken clouds,-18.6500,-173.9833,
20,Atuona,PF,78.24,clear sky,-9.8000,-139.0333,
25,Katsuura,JP,70.11,few clouds,35.1333,140.3000,
26,Tezu,IN,77.16,broken clouds,27.9167,96.1667,
41,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
44,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
46,Ponta Do Sol,PT,70.72,scattered clouds,32.6667,-17.1000,
48,Moerai,PF,76.17,light rain,-22.4333,-151.3333,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df[(hotel_df['Hotel Name'])==""].count()

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df=hotel_df.dropna()
# hotel_df[(hotel_df['Hotel Name'])==""].count()

In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambanglipuro,ID,76.55,overcast clouds,-7.9500,110.2833,Puri Brata
5,Arcata,US,73.20,clear sky,40.8665,-124.0828,"Hampton Inn & Suites Arcata, CA"
12,Neiafu,TO,79.48,broken clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
20,Atuona,PF,78.24,clear sky,-9.8000,-139.0333,Villa Enata
25,Katsuura,JP,70.11,few clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
...,...,...,...,...,...,...,...
646,Tuy Hoa,VN,76.23,broken clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
660,Guhagar,IN,75.45,overcast clouds,17.4842,73.1929,Hotel Seawinds
662,Gidam,IN,72.61,moderate rain,18.9833,81.4000,Dhanshree Lodge And DormItory
668,Kutum,SD,71.10,scattered clouds,14.2000,24.6667,Dibbo areaمنطقة دبو


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(marker_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))